In [1]:
import pandas as pd 
AWM = pd.read_csv('/Users/5036365/Documents/Dravidian Lang Tech Data/misogyny/train-tamil/train.csv')
len(AWM)

1136

In [2]:
AWM_val = pd.read_csv('/Users/5036365/Documents/Dravidian Lang Tech Data/misogyny/dev-tamil/dev.csv')
len(AWM_val)

284

In [3]:
AWM.columns

Index(['image_id', 'labels', 'transcriptions'], dtype='object')

In [25]:
AWM['labels'].value_counts()

labels
0    816
1    285
Name: count, dtype: int64

In [ ]:
AWM = AWM.drop_duplicates(subset=['transcriptions'])
AWM = AWM.reset_index()

In [24]:
AWM['transcriptions'].nunique()

1101

In [26]:
from transformers import AutoTokenizer
 
import torch
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import os

# Split the data into train and test sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(AWM['Text'], AWM['label'], test_size=0.2)
 

train_texts, train_labels,test_texts, test_labels = AWM['transcriptions'], AWM['labels'],AWM_val['transcriptions'], AWM_val['labels']
# Load tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
 
# Tokenize dataset
def preprocess_function(examples):
    return tokenizer(examples.to_list(), truncation=True, padding="max_length", max_length=128)
 
train_encodings = preprocess_function(train_texts)
test_encodings = preprocess_function(test_texts)

In [27]:
# Convert to Hugging Face Dataset format, and include 'labels' in the Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels  # Ensure labels are included
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels  # Ensure labels are included
})


In [28]:
from transformers import AutoModelForSequenceClassification
 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=10,              # Number of epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/1420 [00:00<?, ?it/s]

{'loss': 0.6737, 'grad_norm': 3.0111374855041504, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.07}
{'loss': 0.6699, 'grad_norm': 4.334657669067383, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.14}
{'loss': 0.66, 'grad_norm': 4.714353084564209, 'learning_rate': 3.0000000000000004e-07, 'epoch': 0.21}
{'loss': 0.6623, 'grad_norm': 2.572345495223999, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.28}
{'loss': 0.6641, 'grad_norm': 7.274855136871338, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.35}
{'loss': 0.6687, 'grad_norm': 7.082417964935303, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.42}
{'loss': 0.6612, 'grad_norm': 5.601582050323486, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.49}
{'loss': 0.6541, 'grad_norm': 2.1957602500915527, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.56}
{'loss': 0.6472, 'grad_norm': 2.088592290878296, 'learning_rate': 9.000000000000001e-07, 'epoch': 0.63}
{'loss': 0.6462, 'grad_norm': 2.455136775970459, 'learning_r

  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.639799177646637, 'eval_accuracy': 0.7394366197183099, 'eval_macro_f1': 0.4251012145748988, 'eval_weighted_f1': 0.6286708102868221, 'eval_runtime': 2.8742, 'eval_samples_per_second': 98.809, 'eval_steps_per_second': 12.525, 'epoch': 1.0}
{'loss': 0.6566, 'grad_norm': 7.977200508117676, 'learning_rate': 1.5e-06, 'epoch': 1.06}
{'loss': 0.6512, 'grad_norm': 9.064903259277344, 'learning_rate': 1.6000000000000001e-06, 'epoch': 1.13}
{'loss': 0.6243, 'grad_norm': 18.37721061706543, 'learning_rate': 1.7000000000000002e-06, 'epoch': 1.2}
{'loss': 0.562, 'grad_norm': 8.3906888961792, 'learning_rate': 1.8000000000000001e-06, 'epoch': 1.27}
{'loss': 0.5555, 'grad_norm': 11.768756866455078, 'learning_rate': 1.9000000000000002e-06, 'epoch': 1.34}
{'loss': 0.496, 'grad_norm': 24.788227081298828, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.41}
{'loss': 0.664, 'grad_norm': 184.870361328125, 'learning_rate': 2.1000000000000002e-06, 'epoch': 1.48}
{'loss': 0.5877, 'grad_norm': 15

...

{'eval_loss': 0.6781708002090454, 'eval_accuracy': 0.7535211267605634, 'eval_macro_f1': 0.6389131066550421, 'eval_weighted_f1': 0.7363299237447352, 'eval_runtime': 2.6044, 'eval_samples_per_second': 109.047, 'eval_steps_per_second': 13.823, 'epoch': 8.0}
{'loss': 0.42, 'grad_norm': 36.83182144165039, 'learning_rate': 1.521739130434783e-06, 'epoch': 8.03}
{'loss': 0.2809, 'grad_norm': 26.89448356628418, 'learning_rate': 1.4673913043478264e-06, 'epoch': 8.1}
{'loss': 0.3889, 'grad_norm': 57.375328063964844, 'learning_rate': 1.4130434782608697e-06, 'epoch': 8.17}
{'loss': 0.323, 'grad_norm': 18.257061004638672, 'learning_rate': 1.3586956521739131e-06, 'epoch': 8.24}
{'loss': 0.3049, 'grad_norm': 35.98467254638672, 'learning_rate': 1.3043478260869566e-06, 'epoch': 8.31}
{'loss': 0.3537, 'grad_norm': 46.766746520996094, 'learning_rate': 1.25e-06, 'epoch': 8.38}
{'loss': 0.2098, 'grad_norm': 4.087879657745361, 'learning_rate': 1.1956521739130436e-06, 'epoch': 8.45}
{'loss': 0.2605, 'grad_nor

  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.6663820743560791, 'eval_accuracy': 0.7570422535211268, 'eval_macro_f1': 0.6713789807314987, 'eval_weighted_f1': 0.7517252227962533, 'eval_runtime': 2.3534, 'eval_samples_per_second': 120.675, 'eval_steps_per_second': 15.297, 'epoch': 9.0}
{'loss': 0.2828, 'grad_norm': 43.95911407470703, 'learning_rate': 7.608695652173914e-07, 'epoch': 9.01}
{'loss': 0.2951, 'grad_norm': 32.72504806518555, 'learning_rate': 7.065217391304348e-07, 'epoch': 9.08}
{'loss': 0.3067, 'grad_norm': 26.530223846435547, 'learning_rate': 6.521739130434783e-07, 'epoch': 9.15}
{'loss': 0.2254, 'grad_norm': 6.565260887145996, 'learning_rate': 5.978260869565218e-07, 'epoch': 9.23}
{'loss': 0.3086, 'grad_norm': 62.43011474609375, 'learning_rate': 5.434782608695653e-07, 'epoch': 9.3}
{'loss': 0.3097, 'grad_norm': 7.8002448081970215, 'learning_rate': 4.891304347826088e-07, 'epoch': 9.37}
{'loss': 0.2687, 'grad_norm': 31.50235939025879, 'learning_rate': 4.347826086956522e-07, 'epoch': 9.44}
{'loss': 0.3596,

  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.7094165086746216, 'eval_accuracy': 0.7605633802816901, 'eval_macro_f1': 0.6712962962962963, 'eval_weighted_f1': 0.7533255086071987, 'eval_runtime': 2.4652, 'eval_samples_per_second': 115.203, 'eval_steps_per_second': 14.603, 'epoch': 10.0}
{'train_runtime': 616.9336, 'train_samples_per_second': 18.414, 'train_steps_per_second': 2.302, 'train_loss': 0.45484812452759543, 'epoch': 10.0}


In [14]:
from transformers import AutoTokenizer
 
import torch
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import os
from transformers import BertTokenizer, BertForSequenceClassification

# Split the data into train and test sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(AWM['Text'], AWM['label'], test_size=0.2)
 

train_texts, train_labels,test_texts, test_labels = AWM['transcriptions'], AWM['labels'],AWM_val['transcriptions'], AWM_val['labels']
# Load tokenizer
model_name = "l3cube-pune/tamil-bert"
tokenizer = BertTokenizer.from_pretrained(model_name)
 
# Tokenize dataset
def preprocess_function(examples):
    return tokenizer(examples.to_list(), truncation=True, padding="max_length", max_length=128)
 
train_encodings = preprocess_function(train_texts)
test_encodings = preprocess_function(test_texts)

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

In [15]:
# Convert to Hugging Face Dataset format, and include 'labels' in the Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels  # Ensure labels are included
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels  # Ensure labels are included
})


In [16]:
 
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/tamil-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=10,              # Number of epochs
    per_device_train_batch_size=4,   # Batch size for training
    per_device_eval_batch_size=4,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/2840 [00:00<?, ?it/s]

{'loss': 0.4404, 'grad_norm': 6.413241863250732, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}
{'loss': 0.4689, 'grad_norm': 2.517507791519165, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.07}
{'loss': 0.368, 'grad_norm': 0.858818769454956, 'learning_rate': 3e-06, 'epoch': 0.11}
{'loss': 0.3734, 'grad_norm': 0.8494640588760376, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.14}
{'loss': 0.2918, 'grad_norm': 2.7057077884674072, 'learning_rate': 5e-06, 'epoch': 0.18}
{'loss': 0.48, 'grad_norm': 0.7819114327430725, 'learning_rate': 6e-06, 'epoch': 0.21}
{'loss': 0.3225, 'grad_norm': 2.4913268089294434, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.25}
{'loss': 0.6181, 'grad_norm': 0.8023355603218079, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.28}
{'loss': 0.4943, 'grad_norm': 0.687935471534729, 'learning_rate': 9e-06, 'epoch': 0.32}
{'loss': 0.4017, 'grad_norm': 0.6818622350692749, 'learning_rate': 1e-05, 'epoch': 0.35}
{'loss': 0.3953, 'grad_norm': 1.032

  0%|          | 0/71 [00:00<?, ?it/s]

{'eval_loss': 0.593526303768158, 'eval_accuracy': 0.6936619718309859, 'eval_macro_f1': 0.6679701673049788, 'eval_weighted_f1': 0.7121990966155732, 'eval_runtime': 2.7479, 'eval_samples_per_second': 103.352, 'eval_steps_per_second': 25.838, 'epoch': 1.0}
{'loss': 0.4117, 'grad_norm': 1.148139238357544, 'learning_rate': 2.9e-05, 'epoch': 1.02}
{'loss': 0.4089, 'grad_norm': 42.216129302978516, 'learning_rate': 3e-05, 'epoch': 1.06}
{'loss': 0.4305, 'grad_norm': 0.5749794244766235, 'learning_rate': 3.1e-05, 'epoch': 1.09}
{'loss': 0.6265, 'grad_norm': 2.114341974258423, 'learning_rate': 3.2000000000000005e-05, 'epoch': 1.13}
{'loss': 0.4143, 'grad_norm': 13.98011589050293, 'learning_rate': 3.3e-05, 'epoch': 1.16}
{'loss': 0.439, 'grad_norm': 1.2539098262786865, 'learning_rate': 3.4000000000000007e-05, 'epoch': 1.2}
{'loss': 0.4067, 'grad_norm': 0.8559134006500244, 'learning_rate': 3.5e-05, 'epoch': 1.23}
{'loss': 0.5199, 'grad_norm': 1.3065237998962402, 'learning_rate': 3.6e-05, 'epoch': 1

  0%|          | 0/71 [00:00<?, ?it/s]

...

{'eval_loss': 0.6954302787780762, 'eval_accuracy': 0.7640845070422535, 'eval_macro_f1': 0.6347442172953258, 'eval_weighted_f1': 0.7388287108194806, 'eval_runtime': 2.7488, 'eval_samples_per_second': 103.318, 'eval_steps_per_second': 25.829, 'epoch': 8.0}
{'loss': 0.4587, 'grad_norm': 0.43548423051834106, 'learning_rate': 1.1965811965811967e-05, 'epoch': 8.03}
{'loss': 0.2033, 'grad_norm': 0.44193974137306213, 'learning_rate': 1.1752136752136752e-05, 'epoch': 8.06}
{'loss': 0.4704, 'grad_norm': 0.44843289256095886, 'learning_rate': 1.153846153846154e-05, 'epoch': 8.1}
{'loss': 0.3987, 'grad_norm': 1.4923994541168213, 'learning_rate': 1.1324786324786326e-05, 'epoch': 8.13}
{'loss': 0.2632, 'grad_norm': 3.0547890663146973, 'learning_rate': 1.1111111111111112e-05, 'epoch': 8.17}
{'loss': 0.3894, 'grad_norm': 0.5018665194511414, 'learning_rate': 1.0897435897435898e-05, 'epoch': 8.2}
{'loss': 0.4569, 'grad_norm': 0.49662941694259644, 'learning_rate': 1.0683760683760684e-05, 'epoch': 8.24}
{'

  0%|          | 0/71 [00:00<?, ?it/s]

{'eval_loss': 0.7159718871116638, 'eval_accuracy': 0.7640845070422535, 'eval_macro_f1': 0.6347442172953258, 'eval_weighted_f1': 0.7388287108194806, 'eval_runtime': 2.7965, 'eval_samples_per_second': 101.557, 'eval_steps_per_second': 25.389, 'epoch': 9.0}
{'loss': 0.2008, 'grad_norm': 0.4929856061935425, 'learning_rate': 5.982905982905984e-06, 'epoch': 9.01}
{'loss': 0.2013, 'grad_norm': 0.49437931180000305, 'learning_rate': 5.76923076923077e-06, 'epoch': 9.05}
{'loss': 0.2025, 'grad_norm': 0.40032726526260376, 'learning_rate': 5.555555555555556e-06, 'epoch': 9.08}
{'loss': 0.2685, 'grad_norm': 0.4163222908973694, 'learning_rate': 5.341880341880342e-06, 'epoch': 9.12}
{'loss': 0.3317, 'grad_norm': 0.43761348724365234, 'learning_rate': 5.128205128205128e-06, 'epoch': 9.15}
{'loss': 0.1998, 'grad_norm': 1.7084722518920898, 'learning_rate': 4.914529914529915e-06, 'epoch': 9.19}
{'loss': 0.5418, 'grad_norm': 1.6345511674880981, 'learning_rate': 4.700854700854701e-06, 'epoch': 9.23}
{'loss':

  0%|          | 0/71 [00:00<?, ?it/s]

{'eval_loss': 0.7489249110221863, 'eval_accuracy': 0.7570422535211268, 'eval_macro_f1': 0.6188660896625499, 'eval_weighted_f1': 0.7287605825559209, 'eval_runtime': 2.7498, 'eval_samples_per_second': 103.28, 'eval_steps_per_second': 25.82, 'epoch': 10.0}
{'train_runtime': 811.3114, 'train_samples_per_second': 14.002, 'train_steps_per_second': 3.501, 'train_loss': 0.3846003410891748, 'epoch': 10.0}


In [ ]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=10,              # Number of epochs
    per_device_train_batch_size=2,   # Batch size for training
    per_device_eval_batch_size=2,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/5680 [00:00<?, ?it/s]

{'loss': 0.3253, 'grad_norm': 9.137367248535156, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}
{'loss': 0.4579, 'grad_norm': 3.7114157676696777, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.04}
{'loss': 0.5434, 'grad_norm': 265.4037780761719, 'learning_rate': 3e-06, 'epoch': 0.05}
{'loss': 0.3108, 'grad_norm': 31.866313934326172, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.07}
{'loss': 0.3842, 'grad_norm': 1.8123210668563843, 'learning_rate': 5e-06, 'epoch': 0.09}
{'loss': 0.3337, 'grad_norm': 0.5322614312171936, 'learning_rate': 6e-06, 'epoch': 0.11}
{'loss': 0.339, 'grad_norm': 10.101898193359375, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.12}
{'loss': 0.4126, 'grad_norm': 14.117105484008789, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.14}
{'loss': 0.2356, 'grad_norm': 0.5121042132377625, 'learning_rate': 9e-06, 'epoch': 0.16}
{'loss': 0.4076, 'grad_norm': 0.49489864706993103, 'learning_rate': 1e-05, 'epoch': 0.18}
{'loss': 0.5278, 'grad_norm': 

  0%|          | 0/142 [00:00<?, ?it/s]

{'eval_loss': 0.8192501664161682, 'eval_accuracy': 0.7570422535211268, 'eval_macro_f1': 0.6576938529529932, 'eval_weighted_f1': 0.7460035423468896, 'eval_runtime': 3.4879, 'eval_samples_per_second': 81.424, 'eval_steps_per_second': 40.712, 'epoch': 1.0}
{'loss': 0.152, 'grad_norm': 0.3092503547668457, 'learning_rate': 4.9324324324324325e-05, 'epoch': 1.0}
{'loss': 0.6488, 'grad_norm': 0.2214561253786087, 'learning_rate': 4.922779922779923e-05, 'epoch': 1.02}
{'loss': 0.5544, 'grad_norm': 3.992719888687134, 'learning_rate': 4.913127413127413e-05, 'epoch': 1.04}
{'loss': 0.1642, 'grad_norm': 384.7154846191406, 'learning_rate': 4.903474903474904e-05, 'epoch': 1.06}
{'loss': 1.1704, 'grad_norm': 2403.698974609375, 'learning_rate': 4.893822393822394e-05, 'epoch': 1.07}
{'loss': 0.2502, 'grad_norm': 1.139636516571045, 'learning_rate': 4.884169884169885e-05, 'epoch': 1.09}
{'loss': 1.3168, 'grad_norm': 9.800310134887695, 'learning_rate': 4.874517374517375e-05, 'epoch': 1.11}
{'loss': 0.8823, 

  0%|          | 0/142 [00:00<?, ?it/s]

...

{'eval_loss': 0.8379851579666138, 'eval_accuracy': 0.7640845070422535, 'eval_macro_f1': 0.6196758010033779, 'eval_weighted_f1': 0.7319019954107328, 'eval_runtime': 3.5518, 'eval_samples_per_second': 79.96, 'eval_steps_per_second': 39.98, 'epoch': 8.0}
{'loss': 1.244, 'grad_norm': 4.46653413772583, 'learning_rate': 1.0907335907335908e-05, 'epoch': 8.01}
{'loss': 0.6737, 'grad_norm': 4.436098575592041, 'learning_rate': 1.0810810810810812e-05, 'epoch': 8.03}
{'loss': 0.6839, 'grad_norm': 0.334575355052948, 'learning_rate': 1.0714285714285714e-05, 'epoch': 8.05}
{'loss': 0.5435, 'grad_norm': 0.3271658718585968, 'learning_rate': 1.0617760617760618e-05, 'epoch': 8.06}
{'loss': 0.7109, 'grad_norm': 4.174884796142578, 'learning_rate': 1.0521235521235521e-05, 'epoch': 8.08}
{'loss': 0.8696, 'grad_norm': 4.227121353149414, 'learning_rate': 1.0424710424710425e-05, 'epoch': 8.1}
{'loss': 0.5351, 'grad_norm': 0.36224454641342163, 'learning_rate': 1.0328185328185329e-05, 'epoch': 8.12}
{'loss': 0.82

  0%|          | 0/142 [00:00<?, ?it/s]

{'eval_loss': 0.8263354301452637, 'eval_accuracy': 0.7640845070422535, 'eval_macro_f1': 0.6347442172953258, 'eval_weighted_f1': 0.7388287108194806, 'eval_runtime': 3.5167, 'eval_samples_per_second': 80.756, 'eval_steps_per_second': 40.378, 'epoch': 9.0}
{'loss': 0.2042, 'grad_norm': 0.30264756083488464, 'learning_rate': 5.405405405405406e-06, 'epoch': 9.01}
{'loss': 0.2148, 'grad_norm': 0.3309290111064911, 'learning_rate': 5.308880308880309e-06, 'epoch': 9.03}
{'loss': 0.3471, 'grad_norm': 0.31423282623291016, 'learning_rate': 5.212355212355213e-06, 'epoch': 9.05}
{'loss': 0.3821, 'grad_norm': 0.28381776809692383, 'learning_rate': 5.115830115830116e-06, 'epoch': 9.07}
{'loss': 0.6624, 'grad_norm': 0.3199167251586914, 'learning_rate': 5.019305019305019e-06, 'epoch': 9.08}
{'loss': 0.2036, 'grad_norm': 0.31610390543937683, 'learning_rate': 4.922779922779923e-06, 'epoch': 9.1}
{'loss': 0.3759, 'grad_norm': 0.31107866764068604, 'learning_rate': 4.8262548262548266e-06, 'epoch': 9.12}
{'loss

  0%|          | 0/142 [00:00<?, ?it/s]

{'eval_loss': 0.8944197297096252, 'eval_accuracy': 0.75, 'eval_macro_f1': 0.5586490686628581, 'eval_weighted_f1': 0.6978133823625977, 'eval_runtime': 3.5187, 'eval_samples_per_second': 80.711, 'eval_steps_per_second': 40.356, 'epoch': 10.0}
{'train_runtime': 1350.9498, 'train_samples_per_second': 8.409, 'train_steps_per_second': 4.204, 'train_loss': 0.5189296061592833, 'epoch': 10.0}


In [20]:
from transformers import TrainingArguments, Trainer
 # Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    weighted_f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1
    }

class TrainMetricsLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Epoch: {state.epoch}, Training Metrics: {logs}")

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',     # Evaluate every epoch
    num_train_epochs=10,              # Number of epochs
    per_device_train_batch_size=1,   # Batch size for training
    per_device_eval_batch_size=1,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.05,               # Strength of weight decay
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy='epoch',
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
train_result = trainer.train()

  0%|          | 0/11360 [00:00<?, ?it/s]

{'loss': 0.9365, 'grad_norm': 0.40558308362960815, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.9336, 'grad_norm': 7.835175514221191, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}
{'loss': 0.3506, 'grad_norm': 0.4204024374485016, 'learning_rate': 3e-06, 'epoch': 0.03}
{'loss': 0.3425, 'grad_norm': 0.39777010679244995, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.04}
{'loss': 1.233, 'grad_norm': 0.4082263708114624, 'learning_rate': 5e-06, 'epoch': 0.04}
{'loss': 0.6413, 'grad_norm': 0.40753984451293945, 'learning_rate': 6e-06, 'epoch': 0.05}
{'loss': 0.3515, 'grad_norm': 0.3893161118030548, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.06}
{'loss': 0.9327, 'grad_norm': 7.847845554351807, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.07}
{'loss': 0.6447, 'grad_norm': 0.6113662123680115, 'learning_rate': 9e-06, 'epoch': 0.08}
{'loss': 0.3543, 'grad_norm': 0.5913524031639099, 'learning_rate': 1e-05, 'epoch': 0.09}
{'loss': 0.3434, 'grad_norm'

  0%|          | 0/284 [00:00<?, ?it/s]

{'eval_loss': 0.8373655080795288, 'eval_accuracy': 0.7394366197183099, 'eval_macro_f1': 0.4251012145748988, 'eval_weighted_f1': 0.6286708102868221, 'eval_runtime': 6.0814, 'eval_samples_per_second': 46.7, 'eval_steps_per_second': 46.7, 'epoch': 1.0}
{'loss': 0.945, 'grad_norm': 7.8397536277771, 'learning_rate': 4.7053406998158386e-05, 'epoch': 1.0}
{'loss': 0.9151, 'grad_norm': 0.4541016221046448, 'learning_rate': 4.700736648250461e-05, 'epoch': 1.01}
{'loss': 1.7504, 'grad_norm': 0.5110873579978943, 'learning_rate': 4.6961325966850835e-05, 'epoch': 1.02}
{'loss': 0.5941, 'grad_norm': 0.5363752841949463, 'learning_rate': 4.6915285451197057e-05, 'epoch': 1.03}
{'loss': 1.1337, 'grad_norm': 7.094141960144043, 'learning_rate': 4.6869244935543285e-05, 'epoch': 1.04}
{'loss': 0.3398, 'grad_norm': 0.48281198740005493, 'learning_rate': 4.6823204419889506e-05, 'epoch': 1.05}
{'loss': 0.6233, 'grad_norm': 0.41327935457229614, 'learning_rate': 4.6777163904235734e-05, 'epoch': 1.06}
{'loss': 0.93

  0%|          | 0/284 [00:00<?, ?it/s]

...

  0%|          | 0/284 [00:00<?, ?it/s]

{'eval_loss': 0.8739458322525024, 'eval_accuracy': 0.7394366197183099, 'eval_macro_f1': 0.4251012145748988, 'eval_weighted_f1': 0.6286708102868221, 'eval_runtime': 6.3742, 'eval_samples_per_second': 44.554, 'eval_steps_per_second': 44.554, 'epoch': 6.0}
{'loss': 1.2769, 'grad_norm': 0.3577388525009155, 'learning_rate': 2.0902394106813997e-05, 'epoch': 6.0}
{'loss': 0.9743, 'grad_norm': 0.35103094577789307, 'learning_rate': 2.0856353591160222e-05, 'epoch': 6.01}
{'loss': 0.9593, 'grad_norm': 7.805878162384033, 'learning_rate': 2.0810313075506447e-05, 'epoch': 6.02}
{'loss': 1.5559, 'grad_norm': 0.41830676794052124, 'learning_rate': 2.076427255985267e-05, 'epoch': 6.03}
{'loss': 1.2202, 'grad_norm': 0.4167958199977875, 'learning_rate': 2.0718232044198896e-05, 'epoch': 6.04}
{'loss': 0.6336, 'grad_norm': 0.4288772642612457, 'learning_rate': 2.067219152854512e-05, 'epoch': 6.05}
{'loss': 0.9269, 'grad_norm': 0.409666508436203, 'learning_rate': 2.0626151012891345e-05, 'epoch': 6.06}
{'loss'

  0%|          | 0/284 [00:00<?, ?it/s]

{'eval_loss': 0.8941230177879333, 'eval_accuracy': 0.7394366197183099, 'eval_macro_f1': 0.4251012145748988, 'eval_weighted_f1': 0.6286708102868221, 'eval_runtime': 5.7889, 'eval_samples_per_second': 49.06, 'eval_steps_per_second': 49.06, 'epoch': 7.0}
{'loss': 0.671, 'grad_norm': 8.065064430236816, 'learning_rate': 1.565377532228361e-05, 'epoch': 7.01}
{'loss': 0.3594, 'grad_norm': 8.200859069824219, 'learning_rate': 1.5607734806629834e-05, 'epoch': 7.02}
{'loss': 0.6822, 'grad_norm': 8.445530891418457, 'learning_rate': 1.556169429097606e-05, 'epoch': 7.02}
{'loss': 1.0277, 'grad_norm': 0.3350711166858673, 'learning_rate': 1.5515653775322283e-05, 'epoch': 7.03}
{'loss': 0.7068, 'grad_norm': 0.2917410731315613, 'learning_rate': 1.5469613259668508e-05, 'epoch': 7.04}
{'loss': 0.0357, 'grad_norm': 0.29271399974823, 'learning_rate': 1.5423572744014733e-05, 'epoch': 7.05}
{'loss': 1.0351, 'grad_norm': 8.330621719360352, 'learning_rate': 1.5377532228360957e-05, 'epoch': 7.06}
{'loss': 1.0543

KeyboardInterrupt: 